In [ ]:
import openai
import pandas as pd
import os
import json
from getpass import getpass
import csv
import requests
import time
from openai import OpenAI
import threading
from concurrent.futures import ThreadPoolExecutor


In [ ]:

# Load texts from Excel file
try:
    df_texts = pd.read_excel('99_texts.xlsx', header=None)
    texts = df_texts[2].tolist()
except FileNotFoundError:
    print("Error: Excel file not found.")
    exit(1)
except pd.errors.EmptyDataError:
    print("Error: Excel file is empty or corrupted.")
    exit(1)
except (pd.errors.ParserError, PermissionError) as e:
    print(f"Error reading Excel file: {e}")
    exit(1)

# Load groups from file
groups = []
try:
    with open('C_99_9_2.txt', 'r') as f:
        for line in f:
            groups.append([int(x) for x in line.strip().split()])
except FileNotFoundError:
    print("Error: Groups file not found.")
    exit(1)
except Exception as e:
    print(f"Error reading groups file: {e}")
    exit(1)

# Test mode: process only first k groups
k = 1  # Change k as needed, or comment out this line to process all groups
groups = groups[:k]

# Input API key
api_key = getpass('Enter your OpenAI API key: ')
os.environ["OPENAI_API_KEY"] = api_key
client = OpenAI(api_key=api_key)

# Function to generate prompt
def generate_prompt(group_indices, texts):
    """
    Generates a prompt for the given group of texts.
    """
    prompt = (
        "Найди в текстах ниже \"нетривиальные\" интертекстуальные связи. "
        "\"Нетривиальные\" означают, что они специфические, небанальные, "
        "поразительные, как будто бы один переписывал у другого, они должны выглядеть уникальными. "
        "Это должны быть действительно нетривиальные связи, их может и не быть в этих текстах, "
        "давай ровно столько пунктов, сколько этих связей есть, не надо давать эти связи просто ради количества. "
        "От тебя не требуются интерпретации, пояснения. Ты должен только очень четко, с деталями "
        "(совпадающие или очень похожие слова, образы, ситуации) показывать, в чем состоит связь. "
        "Если есть одинаковые, похожие слова, приводи соответствующие цитаты. "
        "Если связь более общая, очень четко определи, в чем она состоит. Повторяю, эта связь должна быть "
        "настолько близкой, нетривиальной, небанальной, что таких связей между текстами разных авторов обычно не бывает. "
        "Если такую же связь можно провести и с многими другими текстами, на которых ты проходил обучение, о которых ты знаешь, "
        "то эту связь мы считаем тривиальной и приводить ее здесь не надо. Для каждой связи укажи также "
        "\"уровень преднамеренности\" (от 0 до 100). Если такая связь могла у авторов получиться случайно, то значение должно быть "
        "около 0 (если в литературе такие ходы встречаются довольно часто). Если же детали настолько хорошо соответствуют друг другу, "
        "и они сами по себе достаточно специфические, поэтому почти наверняка один автор держит в голове текст другого автора, "
        "когда это пишет, или даже ссылается, то тогда значение должно быть ближе к 100.\n"
        "Ничего не выдумывай и перепроверяй. Запрещается использовать какие-либо ссылки или упоминания авторов, их известные произведения, "
        "цитаты и философские идеи, если они не присутствуют прямо в тексте. Связи должны быть строго выведены из содержания данных текстов.\n"
        "Если нет ни одной действительно нетривиальной, неслучайной связи, напиши: \"Нет ни одной\"."
    )
    for idx in group_indices:
        text_number = idx
        text_content = texts[idx - 1]  # Indexing starts at 0
        prompt += f"\n\n<Текст №{text_number} Начало>\n{text_content}\n<Текст №{text_number} Конец>"
    return prompt

# Function to write results to CSV
def write_result_to_csv(result, output_csv='results.csv'):
    """
    Writes a single result to a CSV file, appending it.
    """
    fieldnames = ['Group', 'Response']
    file_exists = os.path.isfile(output_csv)
    with open(output_csv, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow(result)

# Process each group individually
for idx, group_indices in enumerate(groups, start=1):
    print(f"\nProcessing group {idx}/{len(groups)}: {group_indices}")
    prompt = generate_prompt(group_indices, texts)
    try:
        # Send request to OpenAI API using the client object
        response = client.chat.completions.create(
            model="o1-preview",
            messages=[{"role": "user", "content": prompt}],
            #max_completion_tokens=1500
        )
        response_text = response.choices[0].message.content
    except Exception as e:
        print(f"Error processing group {idx}: {e}")
        response_text = f"Error: {e}"
    # Write result to CSV
    group_str = ' '.join(map(str, group_indices))
    result = {'Group': group_str, 'Response': response_text}
    write_result_to_csv(result, output_csv='results.csv')
    # Optional delay to avoid rate limits
    #time.sleep(1)  # Adjust as needed

print("\nProcessing completed. Results saved to 'results.csv'.")


Enter your OpenAI API key: ··········

Processing group 1/1: [1, 2, 3, 4, 5, 6, 7, 98, 99]

Processing completed. Results saved to 'results.csv'.


In [ ]:
import os
import pandas as pd
import csv
import threading
from getpass import getpass
from concurrent.futures import ThreadPoolExecutor
from openai import OpenAI

# Load texts from Excel file
try:
    df_texts = pd.read_excel('99_texts.xlsx', header=None)
    texts = df_texts[2].tolist()
except FileNotFoundError:
    print("Error: Excel file not found.")
    exit(1)
except pd.errors.EmptyDataError:
    print("Error: Excel file is empty or corrupted.")
    exit(1)
except (pd.errors.ParserError, PermissionError) as e:
    print(f"Error reading Excel file: {e}")
    exit(1)

# Load groups from file
groups = []
try:
    with open('C_99_9_2.txt', 'r') as f:
        for line in f:
            groups.append([int(x) for x in line.strip().split()])
except FileNotFoundError:
    print("Error: Groups file not found.")
    exit(1)
except Exception as e:
    print(f"Error reading groups file: {e}")
    exit(1)

# Test mode: process only first k groups
#k = 60  # Change k as needed, or comment out this line to process all groups
#groups = groups[:k]

# Input API key
api_key = getpass('Enter your OpenAI API key: ')
os.environ["OPENAI_API_KEY"] = api_key
client = OpenAI(api_key=api_key)

# Function to generate prompt
def generate_prompt(group_indices, texts):
    """
    Generates a prompt for the given group of texts.
    """
    prompt = (
        "Найди в текстах ниже \"нетривиальные\" интертекстуальные связи. "
        "\"Нетривиальные\" означают, что они специфические, небанальные, "
        "поразительные, как будто бы один переписывал у другого, они должны выглядеть уникальными. "
        "Это должны быть действительно нетривиальные связи, их может и не быть в этих текстах, "
        "давай ровно столько пунктов, сколько этих связей есть, не надо давать эти связи просто ради количества. "
        "От тебя не требуются интерпретации, пояснения. Ты должен только очень четко, с деталями "
        "(совпадающие или очень похожие слова, образы, ситуации) показывать, в чем состоит связь. "
        "Если есть одинаковые, похожие слова, приводи соответствующие цитаты. "
        "Если связь более общая, очень четко определи, в чем она состоит. Повторяю, эта связь должна быть "
        "настолько близкой, нетривиальной, небанальной, что таких связей между текстами разных авторов обычно не бывает. "
        "Если такую же связь можно провести и с многими другими текстами, на которых ты проходил обучение, о которых ты знаешь, "
        "то эту связь мы считаем тривиальной и приводить ее здесь не надо. Для каждой связи укажи также "
        "\"уровень преднамеренности\" (от 0 до 100). Если такая связь могла у авторов получиться случайно, то значение должно быть "
        "около 0 (если в литературе такие ходы встречаются довольно часто). Если же детали настолько хорошо соответствуют друг другу, "
        "и они сами по себе достаточно специфические, поэтому почти наверняка один автор держит в голове текст другого автора, "
        "когда это пишет, или даже ссылается, то тогда значение должно быть ближе к 100.\n"
        "Ничего не выдумывай и перепроверяй. Запрещается использовать какие-либо ссылки или упоминания авторов, их известные произведения, "
        "цитаты и философские идеи, если они не присутствуют прямо в тексте. Связи должны быть строго выведены из содержания данных текстов.\n"
        "Если нет ни одной действительно нетривиальной, неслучайной связи, напиши: \"Нет ни одной\"."
    )
    for idx in group_indices:
        text_number = idx
        text_content = texts[idx - 1]  # Indexing starts at 0
        prompt += f"\n\n<Текст №{text_number} Начало>\n{text_content}\n<Текст №{text_number} Конец>"
    return prompt

# Function to write results to CSV
def write_result_to_csv(result, output_csv='results.csv'):
    """
    Writes a single result to a CSV file, appending it.
    """
    fieldnames = ['Group', 'Response']
    file_exists = os.path.isfile(output_csv)
    with open(output_csv, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow(result)

# Function to process a single group
def process_group(idx, group_indices, texts, client, lock):
    print(f"\nProcessing group {idx}: {group_indices}")
    prompt = generate_prompt(group_indices, texts)
    try:
        # Send request to OpenAI API using the client object
        response = client.chat.completions.create(
            model="o1-preview",
            messages=[{"role": "user", "content": prompt}],
            #max_completion_tokens=1500
        )
        response_text = response.choices[0].message.content
    except Exception as e:
        print(f"Error processing group {idx}: {e}")
        response_text = f"Error: {e}"
    # Write result to CSV
    group_str = ' '.join(map(str, group_indices))
    result = {'Group': group_str, 'Response': response_text}
    # Write result to CSV with thread-safe access
    with lock:
        write_result_to_csv(result, output_csv='results.csv')

# Create a lock for thread-safe file writing
lock = threading.Lock()

# Use ThreadPoolExecutor to process groups in parallel
with ThreadPoolExecutor(max_workers=30) as executor:
    futures = []
    for idx, group_indices in enumerate(groups, start=1):
        futures.append(executor.submit(process_group, idx, group_indices, texts, client, lock))

    # Wait for all futures to complete
    for future in futures:
        future.result()

print("\nProcessing completed. Results saved to 'results.csv'.")


Enter your OpenAI API key: ··········

Processing group 1: [1, 2, 3, 4, 5, 6, 7, 8, 9]

Processing group 2: [1, 4, 11, 43, 53, 55, 58, 65, 94]

Processing group 3: [1, 5, 15, 27, 33, 49, 71, 73, 81]

Processing group 4: [1, 6, 21, 29, 34, 42, 76, 86, 88]

Processing group 5: [1, 7, 14, 35, 38, 60, 61, 78, 95]

Processing group 6: [1, 8, 10, 17, 44, 70, 74, 77, 99]

Processing group 7: [1, 9, 24, 36, 48, 51, 64, 84, 91]

Processing group 8: [1, 12, 25, 37, 40, 47, 67, 92, 97]

Processing group 9: [1, 13, 18, 26, 39, 54, 83, 85, 89]

Processing group 10: [1, 16, 23, 30, 31, 41, 57, 59, 69]

Processing group 11: [1, 19, 22, 32, 45, 50, 63, 90, 98]

Processing group 12: [1, 20, 46, 56, 72, 80, 87, 93, 96]

Processing group 13: [1, 28, 52, 62, 66, 68, 75, 79, 82]

Processing group 14: [2, 4, 19, 30, 35, 40, 77, 87, 89]
Processing group 15: [2, 5, 12, 44, 54, 56, 59, 66, 95]


Processing group 16: [2, 6, 13, 25, 31, 50, 72, 74, 79]

Processing group 17: [2, 7, 22, 34, 46, 49, 65, 82, 92]

Pr